In [41]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.0/345.0 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.24.0 requires anyio<4,>=3.1.0, but you have anyio 4.7.0 which is incompatible.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.1 which is incompatible.


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
!pip install -U langchain-community -q
!pip install --upgrade openai langchain langchain-openai -q
!pip install chromadb -q
!pip install tiktoken -q
# !pip install openai==1.55.3 httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.7 MB/s eta 0:00:00


In [20]:
import os
import glob
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [37]:
resume_folder = "/content/drive/My Drive/Resume Chatbot/resumes"

json_paths = glob.glob(os.path.join(resume_folder, "*.json"))

text_splitter = CharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

documents = []

In [38]:
def create_resume_text(candidate):
    name = candidate.get("name", "Unknown Candidate")
    profile = candidate.get("profile", "")
    skills = candidate.get("skills", [])
    education = candidate.get("education", [])
    experience = candidate.get("experience", [])
    projects = candidate.get("projects", [])

    skills_text = "Skills: " + ", ".join(skills) if skills else ""
    education_text = "Education:\n" + "\n".join(education) if education else ""

    experience_text = ""
    if experience:
        experience_text = "Experience:\n"
        for exp in experience:
            exp_position = exp.get("position", "")
            exp_company = exp.get("company", "")
            exp_duration = exp.get("duration", "")
            exp_desc = exp.get("description", [])
            exp_desc_text = "\n".join([f"- {d}" for d in exp_desc])
            experience_text += f"{exp_position} at {exp_company} ({exp_duration})\n{exp_desc_text}\n\n"

    projects_text = ""
    if projects:
        projects_text = "Projects:\n"
        for proj in projects:
            proj_title = proj.get("title", "")
            proj_desc = proj.get("description", [])
            proj_desc_text = "\n".join([f"- {d}" for d in proj_desc])
            projects_text += f"{proj_title}:\n{proj_desc_text}\n\n"

    full_text = f"Name: {name}\nProfile: {profile}\n\n{skills_text}\n\n{education_text}\n\n{experience_text}\n{projects_text}"
    return full_text.strip()

In [39]:
for json_path in json_paths:
    print(f"Checking file: {json_path}")
    if not os.path.exists(json_path):
        print(f"File does not exist: {json_path}")
        continue

    with open(json_path, "r", encoding="utf-8") as f:
        candidates = json.load(f)
        print(f"Loaded {len(candidates)} candidates from {json_path}")

    for candidate_idx, candidate in enumerate(candidates, start=1):
        candidate_name = candidate.get("name", "Unknown Candidate")
        resume_text = create_resume_text(candidate)

        if not resume_text.strip():
            print(f"Candidate {candidate_name} from {json_path} has empty resume_text.")
        else:
            print(f"Candidate {candidate_name} from {json_path}: {len(resume_text)} characters of resume text.")

        chunks = text_splitter.split_text(resume_text)
        print(f"Candidate {candidate_name} from {json_path}: Created {len(chunks)} chunks.")

        for i, chunk in enumerate(chunks):
            if not chunk.strip():
                print(f"Candidate {candidate_name}, chunk {i} is empty.")
            else:
                metadata = {
                    "name": candidate_name,
                    "chunk_id": i
                }
                doc = Document(page_content=chunk, metadata=metadata)
                documents.append(doc)

print(f"Total number of documents created: {len(documents)}")

Checking file: /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json
Loaded 5 candidates from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json
Candidate Ethan Carter from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: 1550 characters of resume text.
Candidate Ethan Carter from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: Created 2 chunks.
Candidate Olivia Johnson from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: 1459 characters of resume text.
Candidate Olivia Johnson from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: Created 2 chunks.
Candidate Liam Brooks from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: 1122 characters of resume text.
Candidate Liam Brooks from /content/drive/My Drive/Resume Chatbot/resumes/backend_engineer.json: Created 2 chunks.
Candidate Sophia Patel from /content/drive/My Drive/Resume Chatbot/resumes/backend

In [40]:
#test
print(len(documents))


44


In [41]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
persist_directory = "chroma_db_directory"
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

vectorstore = Chroma.from_documents(
    documents,
    embedding=embeddings,
    collection_name="candidate_resumes",
    persist_directory=persist_directory
)
vectorstore.persist()

In [42]:
recruiter_query = "We need an expert in sales!"

In [43]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
retrieved_docs = retriever.get_relevant_documents(recruiter_query)

llm = ChatOpenAI(model_name='gpt-4', temperature=0)

In [44]:
system_prompt = """You are a helpful assistant that identifies top candidates given a job description.
Given the recruiter's query and the retrieved resume chunks, list the top 3 candidates.
For each candidate, provide:
1. The candidate's name
2. The specific part of their resume (the chunk) that best matches the job description.
Explain why this chunk is relevant to the recruiter's query."""

retrieved_text = ""
for i, doc in enumerate(retrieved_docs, start=1):
    candidate_name = doc.metadata["name"]
    chunk_content = doc.page_content
    retrieved_text += f"Match #{i} - Candidate: {candidate_name}\nResume Chunk:\n{chunk_content}\n\n"

final_prompt = f"""
The recruiter query: "{recruiter_query}"

The following candidate resume sections have been retrieved as the most relevant:

{retrieved_text}

Please respond with the top 3 candidates that best fit the job description.
For each candidate, state their name and show the relevant portion of their resume above.
Explain how that portion aligns with the job requirements.
"""
from langchain.schema import SystemMessage, HumanMessage

response = llm([
    SystemMessage(content=system_prompt),
    HumanMessage(content=final_prompt)
])

print(response.content)

1. Candidate: Jordan Smith
Relevant Resume Chunk: 
Profile: Experienced VP of Sales with a track record of building and leading high-performing sales teams.
Experience:
VP of Sales at Elite Enterprises (Mar 2015 - Present)
- Increased annual revenue from $20 million to $50 million by implementing strategic sales initiatives.
- Recruited and mentored a sales team of 50, fostering a high-performance culture.
- Negotiated partnerships with key accounts, contributing to 40% of annual revenue.
This chunk is relevant because Jordan Smith has a proven track record as a VP of Sales, demonstrating leadership and the ability to significantly increase revenue. His experience in recruiting and mentoring a large sales team also aligns with the job requirements.

2. Candidate: Jordan Smith
Relevant Resume Chunk: 
Profile: Experienced VP of Sales with a track record of building and leading high-performing sales teams.
Experience:
Regional Sales Manager at GrowthPartners (Jan 2013 - Feb 2015)
- Manage